In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt

In [ ]:
# Define the bounding box for China
# These are approximate coordinates - you might need to adjust them
china_bbox = [73.5, 18.0, 135.0, 53.5]  # [min_lon, min_lat, max_lon, max_lat]

def download_power_infrastructure(bbox, tags, filename=None):
    """
    Download power infrastructure data from OpenStreetMap within a bounding box.
    
    Parameters:
    -----------
    bbox : list
        Bounding box [min_lon, min_lat, max_lon, max_lat]
    tags : dict
        OSM tags to filter for
    filename : str, optional
        If provided, saves the data to this filename (GeoJSON format)
    
    Returns:
    --------
    gdf : GeoDataFrame
        Power infrastructure data
    """
    print(f"Downloading data with tags: {tags}")
    
    # Create a polygon from the bounding box
    bbox_polygon = box(*bbox)
    
    # Download the data using OSMnx
    gdf = ox.features_from_polygon(bbox_polygon, tags)
    
    print(f"Downloaded {len(gdf)} features")
    
    # Save to file if filename is provided
    if filename:
        gdf.to_file(filename, driver='GeoJSON')
        print(f"Saved data to {filename}")
    
    return gdf

# Define tags for power lines
# For high voltage lines, we want power=line with voltage > 35000
power_line_tags = {
    'power': 'line',
}

# Define tags for transmission towers
power_tower_tags = {
    'power': 'tower',
}

# Download power lines
power_lines = download_power_infrastructure(
    china_bbox, 
    power_line_tags, 
    'china_power_lines.geojson'
)

# Download power towers
power_towers = download_power_infrastructure(
    china_bbox, 
    power_tower_tags, 
    'china_power_towers.geojson'
)

# Filter for high voltage lines (> 35 kV)
if 'voltage' in power_lines.columns:
    # Convert voltage to numeric, handling multiple values
    def extract_max_voltage(voltage_str):
        if not isinstance(voltage_str, str):
            return 0
        # Split by semicolon or comma and convert to integers where possible
        voltages = []
        for v in voltage_str.replace(',', ';').split(';'):
            try:
                voltages.append(int(v.strip()))
            except ValueError:
                pass
        return max(voltages) if voltages else 0
    
    power_lines['max_voltage'] = power_lines['voltage'].apply(extract_max_voltage)
    high_voltage_lines = power_lines[power_lines['max_voltage'] > 35000]
    high_voltage_lines.to_file('china_high_voltage_lines.geojson', driver='GeoJSON')
    print(f"Filtered {len(high_voltage_lines)} high voltage lines (> 35 kV)")

# Optional: Basic visualization
def plot_power_infrastructure(lines_gdf, towers_gdf=None, figsize=(15, 15)):
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot lines
    if not lines_gdf.empty:
        lines_gdf.plot(ax=ax, color='red', linewidth=0.5)
    
    # Plot towers
    if towers_gdf is not None and not towers_gdf.empty:
        towers_gdf.plot(ax=ax, color='blue', markersize=1)
    
    ax.set_title('Power Infrastructure in China')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('china_power_infrastructure.png', dpi=300)
    plt.show()

# Try to plot (may be slow or fail with large datasets)
try:
    if 'max_voltage' in locals():
        plot_power_infrastructure(high_voltage_lines, power_towers)
    else:
        plot_power_infrastructure(power_lines, power_towers)
    print("Created visualization")
except Exception as e:
    print(f"Visualization failed: {e}")

print("Processing complete.")